In [1]:
import datetime
import warnings
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

import xgboost
from xgboost import XGBRegressor
from xgboost import plot_importance

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler

%matplotlib inline
sns.set(style="darkgrid")
pd.set_option('display.float_format', lambda x: '%.2f' % x)
warnings.filterwarnings("ignore")

from pandas.testing import assert_frame_equal

import lightgbm as lgb
from lightgbm import LGBMRegressor

from math import isnan


from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split

from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb



In [117]:
train = pd.read_csv(r'C:\Users\pc\Desktop\Data Science Folder\Infor\training_set.csv')
test = pd.read_csv(r'C:\Users\pc\Desktop\Data Science Folder\Infor\testing_set.csv')
submission = pd.read_csv(r'C:\Users\pc\Desktop\Data Science Folder\Infor\submission_sample.csv')


In [118]:
capacities = pd.read_csv(r'C:\Users\pc\Desktop\Data Science Folder\Infor\hotels_capacities.csv')
capacities

,Hotel;Capacity
0,1;23
1,2;251
2,3;123
3,4;51


In [119]:
capacities['Hotel_id'] = capacities['Hotel;Capacity'].str.split(';').map(lambda x: x[0].strip())
capacities['Capacity'] = capacities['Hotel;Capacity'].str.split(';').map(lambda x: x[1].strip())
capacities = capacities.drop('Hotel;Capacity', axis=1)
capacities


,Hotel_id,Capacity
0,1,23
1,2,251
2,3,123
3,4,51


In [6]:
print('\n******* train ********\n', 'shape: ', train.shape, '\n', train.columns, '\n',\
      train.describe(), '\n', train.isnull().sum() )

print('\n******* test ********\n', 'shape: ', test.shape, '\n', test.columns, '\n', \
      test.describe(), '\n', test.isnull().sum() )

print('\n******* submission ********\n', 'shape: ', submission.shape, '\n', submission.columns, '\n', \
      submission.describe(), '\n', submission.isnull().sum() )


******* train ********
 shape:  (356720, 37) 
 Index(['Row_id', 'Hotel_id', 'Date_id', 'Date', 'LT', 'BKG_0', 'Revenue_0',
       'HotelPrice_0', 'PriceComp1_0', 'PriceComp2_0', 'PriceComp3_0',
       'PriceComp4_0', 'BKG_1', 'Revenue_1', 'BKG_7', 'Revenue_7', 'BKG_14',
       'Revenue_14', 'HotelPrice_14', 'PriceComp1_14', 'PriceComp2_14',
       'PriceComp3_14', 'PriceComp4_14', 'BKG_21', 'Revenue_21',
       'HotelPrice_21', 'PriceComp1_21', 'PriceComp2_21', 'PriceComp3_21',
       'PriceComp4_21', 'BKG_28', 'Revenue_28', 'HotelPrice_28',
       'PriceComp1_28', 'PriceComp2_28', 'PriceComp3_28', 'PriceComp4_28'],
      dtype='object') 
           Row_id  Hotel_id   Date_id        LT     BKG_0  Revenue_0  \
count  356720.00 356720.00 356720.00 356720.00 356720.00  356720.00   
mean  2783126.14      2.60  43380.50    184.44      7.64     633.84   
std   1212997.91      1.11    145.13    104.71     18.98    1670.80   
min   1000001.00      1.00  43060.00      0.00      0.00       0.00

In [6]:
print( train.columns )
print( test.columns )

Index(['Row_id', 'Hotel_id', 'Date_id', 'Date', 'LT', 'BKG_0', 'Revenue_0',
       'HotelPrice_0', 'PriceComp1_0', 'PriceComp2_0', 'PriceComp3_0',
       'PriceComp4_0', 'BKG_1', 'Revenue_1', 'BKG_7', 'Revenue_7', 'BKG_14',
       'Revenue_14', 'HotelPrice_14', 'PriceComp1_14', 'PriceComp2_14',
       'PriceComp3_14', 'PriceComp4_14', 'BKG_21', 'Revenue_21',
       'HotelPrice_21', 'PriceComp1_21', 'PriceComp2_21', 'PriceComp3_21',
       'PriceComp4_21', 'BKG_28', 'Revenue_28', 'HotelPrice_28',
       'PriceComp1_28', 'PriceComp2_28', 'PriceComp3_28', 'PriceComp4_28'],
      dtype='object')
Index(['Row_id', 'Hotel_id', 'Date_id', 'Date', 'LT', 'BKG_0', 'Revenue_0',
       'PriceComp1_0', 'PriceComp2_0', 'PriceComp3_0', 'PriceComp4_0', 'BKG_1',
       'Revenue_1', 'BKG_7', 'Revenue_7', 'BKG_14', 'Revenue_14',
       'HotelPrice_14', 'PriceComp1_14', 'PriceComp2_14', 'PriceComp3_14',
       'PriceComp4_14', 'BKG_21', 'Revenue_21', 'HotelPrice_21',
       'PriceComp1_21', 'PriceComp2_21'

In [7]:
train.shape

(356720, 37)

In [120]:
for index,row in train.iterrows():
    if (( row['HotelPrice_0'] == 0)&( row['BKG_0']!= 0)):
        #row['HotelPrice_0']= row['Revenue_0']//row['BKG_0']
        train.set_value(index,'HotelPrice_0',row['Revenue_0']//row['BKG_0'])

        


In [54]:
train[['BKG_0','Revenue_0', 'HotelPrice_0']][train['HotelPrice_0']==0 ]

,BKG_0,Revenue_0,HotelPrice_0
6,0.00,0.00,0.00
9,0.00,0.00,0.00
13,0.00,0.00,0.00
32,0.00,0.00,0.00
35,0.00,0.00,0.00
40,0.00,0.00,0.00
47,0.00,0.00,0.00
52,0.00,0.00,0.00
53,0.00,0.00,0.00
72,0.00,0.00,0.00


In [121]:
for index,row in train.iterrows():
    if (( row['HotelPrice_14'] == 0)&( row['BKG_14']!= 0)):
        #row['HotelPrice_14']= row['Revenue_14']//row['BKG_14']
        train.set_value(index,'HotelPrice_14',row['Revenue_14']//row['BKG_14'])


In [122]:
for index,row in train.iterrows():
    if (( row['HotelPrice_21'] == 0)&( row['BKG_21']!= 0)):
        #row['HotelPrice_21']= row['Revenue_21']//row['BKG_21']
        train.set_value(index,'HotelPrice_21',row['Revenue_21']//row['BKG_21'])


In [123]:
for index,row in train.iterrows():
    if (( row['HotelPrice_28'] == 0)&( row['BKG_28']!= 0)):
        #row['HotelPrice_28']= row['Revenue_28']//row['BKG_28']
        train.set_value(index,'HotelPrice_28',row['Revenue_28']//row['BKG_28'])


In [124]:
y_train = train['HotelPrice_0']
train = train.drop('HotelPrice_0', axis=1)

In [125]:
ntrain = train.shape[0]
ntest = test.shape[0]

In [15]:
train.shape

(356720, 36)

In [126]:
train = train.drop('Row_id', axis=1)
test_id = test['Row_id']
test = test.drop('Row_id', axis=1)


In [127]:
all_data = pd.concat((train, test)).reset_index(drop=True)
all_data

,Hotel_id,Date_id,Date,LT,BKG_0,Revenue_0,PriceComp1_0,PriceComp2_0,PriceComp3_0,PriceComp4_0,...,PriceComp2_21,PriceComp3_21,PriceComp4_21,BKG_28,Revenue_28,HotelPrice_28,PriceComp1_28,PriceComp2_28,PriceComp3_28,PriceComp4_28
0,1,43354,11/09/2018,37,2.80,310.47,175.43,254.68,169.94,200.16,...,265.55,185.22,223.66,2.71,305.81,241.82,188.60,259.64,186.01,222.80
1,1,43319,07/08/2018,255,1.83,267.32,179.59,216.65,159.49,251.33,...,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00
2,1,43493,28/01/2019,298,0.00,0.00,106.84,155.95,155.89,246.87,...,168.45,158.10,246.58,0.00,0.00,173.95,102.82,205.64,161.21,260.02
3,1,43371,28/09/2018,220,2.75,498.15,0.00,259.40,235.57,252.52,...,267.35,237.76,250.31,1.93,348.41,180.54,0.00,220.56,229.74,263.44
4,1,43387,14/10/2018,136,0.93,178.42,186.09,244.76,191.40,265.91,...,246.17,181.92,272.82,0.91,175.16,179.20,180.83,248.96,188.42,260.72
5,1,43521,25/02/2019,325,0.00,0.00,105.79,177.75,161.48,251.68,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,1,43430,26/11/2018,360,0.00,0.00,0.00,0.00,0.00,0.00,...,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00
7,1,43324,12/08/2018,37,11.20,1809.55,267.57,283.32,14.06,282.84,...,272.45,3.78,249.34,11.29,1819.69,217.18,259.40,278.36,2.99,250.20
8,1,43311,30/07/2018,94,5.48,989.52,193.54,266.24,208.32,214.72,...,229.99,199.28,210.91,5.62,941.51,184.56,193.07,264.66,183.75,225.11
9,1,43600,15/05/2019,348,0.00,0.00,0.00,0.00,0.00,0.00,...,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00


In [128]:
all_data.replace(-1, 0 , inplace=True)

In [129]:
capacities['Hotel_id'] = capacities['Hotel_id'].astype(np.int64)
capacities['Capacity'] = capacities['Capacity'].astype(np.int64)
capacities.dtypes

Hotel_id    int64
Capacity    int64
dtype: object

In [130]:
all_data = all_data.merge(capacities, how='left', on='Hotel_id')
all_data

,Hotel_id,Date_id,Date,LT,BKG_0,Revenue_0,PriceComp1_0,PriceComp2_0,PriceComp3_0,PriceComp4_0,...,PriceComp3_21,PriceComp4_21,BKG_28,Revenue_28,HotelPrice_28,PriceComp1_28,PriceComp2_28,PriceComp3_28,PriceComp4_28,Capacity
0,1,43354,11/09/2018,37,2.80,310.47,175.43,254.68,169.94,200.16,...,185.22,223.66,2.71,305.81,241.82,188.60,259.64,186.01,222.80,23
1,1,43319,07/08/2018,255,1.83,267.32,179.59,216.65,159.49,251.33,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,23
2,1,43493,28/01/2019,298,0.00,0.00,106.84,155.95,155.89,246.87,...,158.10,246.58,0.00,0.00,173.95,102.82,205.64,161.21,260.02,23
3,1,43371,28/09/2018,220,2.75,498.15,0.00,259.40,235.57,252.52,...,237.76,250.31,1.93,348.41,180.54,0.00,220.56,229.74,263.44,23
4,1,43387,14/10/2018,136,0.93,178.42,186.09,244.76,191.40,265.91,...,181.92,272.82,0.91,175.16,179.20,180.83,248.96,188.42,260.72,23
5,1,43521,25/02/2019,325,0.00,0.00,105.79,177.75,161.48,251.68,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,23
6,1,43430,26/11/2018,360,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,23
7,1,43324,12/08/2018,37,11.20,1809.55,267.57,283.32,14.06,282.84,...,3.78,249.34,11.29,1819.69,217.18,259.40,278.36,2.99,250.20,23
8,1,43311,30/07/2018,94,5.48,989.52,193.54,266.24,208.32,214.72,...,199.28,210.91,5.62,941.51,184.56,193.07,264.66,183.75,225.11,23
9,1,43600,15/05/2019,348,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,23


In [108]:
all_data['Date'] = pd.to_datetime( all_data.Date, format="%d/%m/%Y") # year in upper case



In [109]:
all_data['month'] = all_data['Date'].dt.month
all_data['year'] = all_data['Date'].dt.year

all_data.head()

,Hotel_id,Date_id,Date,LT,BKG_0,Revenue_0,PriceComp1_0,PriceComp2_0,PriceComp3_0,PriceComp4_0,...,BKG_28,Revenue_28,HotelPrice_28,PriceComp1_28,PriceComp2_28,PriceComp3_28,PriceComp4_28,Capacity,month,year
0,1,43354,2018-09-11,37,2.80,310.47,175.43,254.68,169.94,200.16,...,2.71,305.81,241.82,188.60,259.64,186.01,222.80,23,9,2018
1,1,43319,2018-08-07,255,1.83,267.32,179.59,216.65,159.49,251.33,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,23,8,2018
2,1,43493,2019-01-28,298,0.00,0.00,106.84,155.95,155.89,246.87,...,0.00,0.00,173.95,102.82,205.64,161.21,260.02,23,1,2019
3,1,43371,2018-09-28,220,2.75,498.15,0.00,259.40,235.57,252.52,...,1.93,348.41,180.54,0.00,220.56,229.74,263.44,23,9,2018
4,1,43387,2018-10-14,136,0.93,178.42,186.09,244.76,191.40,265.91,...,0.91,175.16,179.20,180.83,248.96,188.42,260.72,23,10,2018


In [110]:
all_data = all_data[['Hotel_id', 'year', 'month', 'Date_id', 'Date', 'LT', 'BKG_0', 'Revenue_0',
       'PriceComp1_0', 'PriceComp2_0', 'PriceComp3_0', 'PriceComp4_0', 'BKG_1',
       'Revenue_1', 'BKG_7', 'Revenue_7', 'BKG_14', 'Revenue_14',
       'HotelPrice_14', 'PriceComp1_14', 'PriceComp2_14', 'PriceComp3_14',
       'PriceComp4_14', 'BKG_21', 'Revenue_21', 'HotelPrice_21',
       'PriceComp1_21', 'PriceComp2_21', 'PriceComp3_21', 'PriceComp4_21',
       'BKG_28', 'Revenue_28', 'HotelPrice_28', 'PriceComp1_28',
       'PriceComp2_28', 'PriceComp3_28', 'PriceComp4_28', 'Capacity']]

In [111]:
all_data = all_data.drop('Date', axis=1)
all_data

,Hotel_id,year,month,Date_id,LT,BKG_0,Revenue_0,PriceComp1_0,PriceComp2_0,PriceComp3_0,...,PriceComp3_21,PriceComp4_21,BKG_28,Revenue_28,HotelPrice_28,PriceComp1_28,PriceComp2_28,PriceComp3_28,PriceComp4_28,Capacity
0,1,2018,9,43354,37,2.80,310.47,175.43,254.68,169.94,...,185.22,223.66,2.71,305.81,241.82,188.60,259.64,186.01,222.80,23
1,1,2018,8,43319,255,1.83,267.32,179.59,216.65,159.49,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,23
2,1,2019,1,43493,298,0.00,0.00,106.84,155.95,155.89,...,158.10,246.58,0.00,0.00,173.95,102.82,205.64,161.21,260.02,23
3,1,2018,9,43371,220,2.75,498.15,0.00,259.40,235.57,...,237.76,250.31,1.93,348.41,180.54,0.00,220.56,229.74,263.44,23
4,1,2018,10,43387,136,0.93,178.42,186.09,244.76,191.40,...,181.92,272.82,0.91,175.16,179.20,180.83,248.96,188.42,260.72,23
5,1,2019,2,43521,325,0.00,0.00,105.79,177.75,161.48,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,23
6,1,2018,11,43430,360,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,23
7,1,2018,8,43324,37,11.20,1809.55,267.57,283.32,14.06,...,3.78,249.34,11.29,1819.69,217.18,259.40,278.36,2.99,250.20,23
8,1,2018,7,43311,94,5.48,989.52,193.54,266.24,208.32,...,199.28,210.91,5.62,941.51,184.56,193.07,264.66,183.75,225.11,23
9,1,2019,5,43600,348,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,23


# group by

In [86]:
"""
L = ['BKG_1', 'Revenue_1' ] # contains columns having negative values

for col in L:
    
    all_data.loc[ all_data[col]<0, col] = np.nan

    gb = all_data.groupby(by=['Hotel_id', 'year', 'month'], as_index=False)[col].mean()

    for i in range(len(all_data)):
        if ( isnan( all_data[col][i]) ):
            all_data[col][i] = gb[ (gb['Hotel_id'] == all_data['Hotel_id'][i]) & ( gb['year'] == all_data['year'][i]) & ( gb['month'] == all_data['month'][i]) ][col]

"""

"\nL = ['BKG_1', 'Revenue_1' ] # contains columns having negative values\n\nfor col in L:\n    \n    all_data.loc[ all_data[col]<0, col] = np.nan\n\n    gb = all_data.groupby(by=['Hotel_id', 'year', 'month'], as_index=False)[col].mean()\n\n    for i in range(len(all_data)):\n        if ( isnan( all_data[col][i]) ):\n            all_data[col][i] = gb[ (gb['Hotel_id'] == all_data['Hotel_id'][i]) & ( gb['year'] == all_data['year'][i]) & ( gb['month'] == all_data['month'][i]) ][col]\n\n"

In [87]:
"""
L = ['BKG_7', 'Revenue_7', 'BKG_14', 'Revenue_14',
       'HotelPrice_14', 'PriceComp1_14', 'PriceComp2_14', 'PriceComp3_14',
       'PriceComp4_14', 'BKG_21', 'Revenue_21', 'HotelPrice_21',
       'PriceComp1_21', 'PriceComp2_21', 'PriceComp3_21', 'PriceComp4_21',
       'BKG_28', 'Revenue_28', 'HotelPrice_28', 'PriceComp1_28',
       'PriceComp2_28', 'PriceComp3_28', 'PriceComp4_28'] # contains columns having negative values

for col in L:
    
    all_data.loc[ all_data[col]<0, col] = np.nan

    gb = all_data.groupby(by=['Hotel_id', 'year', 'month'], as_index=False)[col].mean()

    for i in range(len(all_data)):
        if ( isnan( all_data[col][i]) ):
            all_data[col][i] = gb[ (gb['Hotel_id'] == all_data['Hotel_id'][i]) & ( gb['year'] == all_data['year'][i]) & ( gb['month'] == all_data['month'][i]) ][col]

"""

"\nL = ['BKG_7', 'Revenue_7', 'BKG_14', 'Revenue_14',\n       'HotelPrice_14', 'PriceComp1_14', 'PriceComp2_14', 'PriceComp3_14',\n       'PriceComp4_14', 'BKG_21', 'Revenue_21', 'HotelPrice_21',\n       'PriceComp1_21', 'PriceComp2_21', 'PriceComp3_21', 'PriceComp4_21',\n       'BKG_28', 'Revenue_28', 'HotelPrice_28', 'PriceComp1_28',\n       'PriceComp2_28', 'PriceComp3_28', 'PriceComp4_28'] # contains columns having negative values\n\nfor col in L:\n    \n    all_data.loc[ all_data[col]<0, col] = np.nan\n\n    gb = all_data.groupby(by=['Hotel_id', 'year', 'month'], as_index=False)[col].mean()\n\n    for i in range(len(all_data)):\n        if ( isnan( all_data[col][i]) ):\n            all_data[col][i] = gb[ (gb['Hotel_id'] == all_data['Hotel_id'][i]) & ( gb['year'] == all_data['year'][i]) & ( gb['month'] == all_data['month'][i]) ][col]\n\n"

In [20]:
all_data.shape

(509602, 37)

# negative values =) nan, nan=) mean

In [89]:
L = ['BKG_1',
       'Revenue_1', 'BKG_7', 'Revenue_7', 'BKG_14', 'Revenue_14',
       'HotelPrice_14', 'PriceComp1_14', 'PriceComp2_14', 'PriceComp3_14',
       'PriceComp4_14', 'BKG_21', 'Revenue_21', 'HotelPrice_21',
       'PriceComp1_21', 'PriceComp2_21', 'PriceComp3_21', 'PriceComp4_21',
       'BKG_28', 'Revenue_28', 'HotelPrice_28', 'PriceComp1_28',
       'PriceComp2_28', 'PriceComp3_28', 'PriceComp4_28'] # contains columns having negative values

for col in L:
    
    all_data.loc[ all_data[col]<0, col] = np.nan

In [26]:
all_data[ all_data < 0 ].count()


Hotel_id            0
year                0
month               0
Date_id             0
LT                  0
BKG_0               0
Revenue_0           0
PriceComp1_0        0
PriceComp2_0        0
PriceComp3_0        0
PriceComp4_0        0
BKG_1              75
Revenue_1           3
BKG_7             513
Revenue_7          10
BKG_14            995
Revenue_14         21
HotelPrice_14     100
PriceComp1_14      86
PriceComp2_14      83
PriceComp3_14      73
PriceComp4_14      85
BKG_21           1428
Revenue_21         26
HotelPrice_21     139
PriceComp1_21     134
PriceComp2_21     104
PriceComp3_21     113
PriceComp4_21     105
BKG_28           1781
Revenue_28         36
HotelPrice_28     151
PriceComp1_28     166
PriceComp2_28     148
PriceComp3_28     156
PriceComp4_28     140
Capacity            0
dtype: int64

In [ ]:
all_data = all_data.fillna( all_data.mean() )

In [92]:
all_data.isnull().sum()

Row_id           0
Hotel_id         0
year             0
month            0
Date_id          0
LT               0
BKG_0            0
Revenue_0        0
PriceComp1_0     0
PriceComp2_0     0
PriceComp3_0     0
PriceComp4_0     0
BKG_1            0
Revenue_1        0
BKG_7            0
Revenue_7        0
BKG_14           0
Revenue_14       0
HotelPrice_14    0
PriceComp1_14    0
PriceComp2_14    0
PriceComp3_14    0
PriceComp4_14    0
BKG_21           0
Revenue_21       0
HotelPrice_21    0
PriceComp1_21    0
PriceComp2_21    0
PriceComp3_21    0
PriceComp4_21    0
BKG_28           0
Revenue_28       0
HotelPrice_28    0
PriceComp1_28    0
PriceComp2_28    0
PriceComp3_28    0
PriceComp4_28    0
Capacity         0
dtype: int64

In [17]:
all_data['Revenue_unit'] = all_data['Revenue_0'] // all_data['Capacity']

all_data['Revenue_unit'].fillna(0, inplace=True)
all_data

,Hotel_id,Date_id,LT,BKG_0,Revenue_0,PriceComp1_0,PriceComp2_0,PriceComp3_0,PriceComp4_0,BKG_1,...,Revenue_28,HotelPrice_28,PriceComp1_28,PriceComp2_28,PriceComp3_28,PriceComp4_28,Capacity,month,year,Revenue_unit
0,1,43354,37,2.80,310.47,175.43,254.68,169.94,200.16,2.92,...,305.81,241.82,188.60,259.64,186.01,222.80,23,9,2018,13.00
1,1,43319,255,1.83,267.32,179.59,216.65,159.49,251.33,1.97,...,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,23,8,2018,11.00
2,1,43493,298,0.00,0.00,106.84,155.95,155.89,246.87,0.00,...,0.00,173.95,102.82,205.64,161.21,260.02,23,1,2019,-0.00
3,1,43371,220,2.75,498.15,0.00,259.40,235.57,252.52,2.93,...,348.41,180.54,0.00,220.56,229.74,263.44,23,9,2018,21.00
4,1,43387,136,0.93,178.42,186.09,244.76,191.40,265.91,0.96,...,175.16,179.20,180.83,248.96,188.42,260.72,23,10,2018,7.00
5,1,43521,325,0.00,0.00,105.79,177.75,161.48,251.68,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,23,2,2019,-0.00
6,1,43430,360,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,23,11,2018,-0.00
7,1,43324,37,11.20,1809.55,267.57,283.32,14.06,282.84,11.08,...,1819.69,217.18,259.40,278.36,2.99,250.20,23,8,2018,78.00
8,1,43311,94,5.48,989.52,193.54,266.24,208.32,214.72,5.76,...,941.51,184.56,193.07,264.66,183.75,225.11,23,7,2018,43.00
9,1,43600,348,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,23,5,2019,-0.00


# min_0 , max_0

In [85]:
all_data3= all_data.copy()


In [86]:
all_data3= all_data.copy()
all_data3['min_0'] = all_data3[['PriceComp1_0', 'PriceComp2_0', 'PriceComp3_0', 'PriceComp4_0']].min(axis=1)
all_data3['max_0'] = all_data3[['PriceComp1_0', 'PriceComp2_0', 'PriceComp3_0', 'PriceComp4_0']].max(axis=1)

all_data3['min_14'] = all_data3[['PriceComp1_14', 'PriceComp2_14', 'PriceComp3_14', 'PriceComp4_14']].min(axis=1)
all_data3['max_14'] = all_data3[['PriceComp1_14', 'PriceComp2_14', 'PriceComp3_14', 'PriceComp4_14']].max(axis=1)

all_data3['min_21'] = all_data3[['PriceComp1_21', 'PriceComp2_21', 'PriceComp3_21', 'PriceComp4_21']].min(axis=1)
all_data3['max_21'] = all_data3[['PriceComp1_21', 'PriceComp2_21', 'PriceComp3_21', 'PriceComp4_21']].max(axis=1)

all_data3['min_28'] = all_data3[['PriceComp1_28', 'PriceComp2_28', 'PriceComp3_28', 'PriceComp4_28']].min(axis=1)
all_data3['max_28'] = all_data3[['PriceComp1_28', 'PriceComp2_28', 'PriceComp3_28', 'PriceComp4_28']].max(axis=1)




"""
all_data3['min_14'] = 0
all_data3['max_14'] = 0
all_data3['min_21'] = 0
all_data3['max_21'] = 0
all_data3['min_28'] = 0
all_data3['max_28'] = 0


for i in range(len(all_data3)):
    all_data3['min_0'][i] = min( all_data3['PriceComp1_0'][i], all_data3['PriceComp2_0'][i], all_data3['PriceComp3_0'][i],\
                                all_data3['PriceComp4_0'][i])
    all_data3['max_0'][i] = max( all_data3['PriceComp1_0'][i], all_data3['PriceComp2_0'][i], all_data3['PriceComp3_0'][i],\
                            all_data3['PriceComp4_0'][i])

    all_data3['min_14'][i] = min( all_data3['PriceComp1_14'][i], all_data3['PriceComp2_14'][i], all_data3['PriceComp3_14'][i],\
                                all_data3['PriceComp4_14'][i])
    all_data3['max_14'][i] = max( all_data3['PriceComp1_14'][i], all_data3['PriceComp2_14'][i], all_data3['PriceComp3_14'][i],\
                                all_data3['PriceComp4_14'][i])
    all_data3['min_21'][i] = min( all_data3['PriceComp1_21'][i], all_data3['PriceComp2_21'][i], all_data3['PriceComp3_21'][i],\
                                all_data3['PriceComp4_21'][i])
    all_data3['max_21'][i] = max( all_data3['PriceComp1_21'][i], all_data3['PriceComp2_21'][i], all_data3['PriceComp3_21'][i],\
                                all_data3['PriceComp4_21'][i])
    all_data3['min_28'][i] = min( all_data3['PriceComp1_28'][i], all_data3['PriceComp2_28'][i], all_data3['PriceComp3_28'][i],\
                                all_data3['PriceComp4_28'][i])
    all_data3['max_28'][i] = max( all_data3['PriceComp1_28'][i], all_data3['PriceComp2_28'][i], all_data3['PriceComp3_28'][i],\
                                all_data3['PriceComp4_28'][i])
"""

"\nall_data3['min_14'] = 0\nall_data3['max_14'] = 0\nall_data3['min_21'] = 0\nall_data3['max_21'] = 0\nall_data3['min_28'] = 0\nall_data3['max_28'] = 0\n\n\nfor i in range(len(all_data3)):\n    all_data3['min_0'][i] = min( all_data3['PriceComp1_0'][i], all_data3['PriceComp2_0'][i], all_data3['PriceComp3_0'][i],                                all_data3['PriceComp4_0'][i])\n    all_data3['max_0'][i] = max( all_data3['PriceComp1_0'][i], all_data3['PriceComp2_0'][i], all_data3['PriceComp3_0'][i],                            all_data3['PriceComp4_0'][i])\n\n    all_data3['min_14'][i] = min( all_data3['PriceComp1_14'][i], all_data3['PriceComp2_14'][i], all_data3['PriceComp3_14'][i],                                all_data3['PriceComp4_14'][i])\n    all_data3['max_14'][i] = max( all_data3['PriceComp1_14'][i], all_data3['PriceComp2_14'][i], all_data3['PriceComp3_14'][i],                                all_data3['PriceComp4_14'][i])\n    all_data3['min_21'][i] = min( all_data3['PriceComp1_21'][i

In [87]:
all_data3 = all_data3.drop(['PriceComp1_0', 'PriceComp2_0', 'PriceComp3_0', 'PriceComp4_0', \
                           'PriceComp1_14', 'PriceComp2_14', 'PriceComp3_14', 'PriceComp4_14', \
                           'PriceComp1_21', 'PriceComp2_21', 'PriceComp3_21', 'PriceComp4_21', \
                            'PriceComp1_28', 'PriceComp2_28', 'PriceComp3_28', 'PriceComp4_28'], axis=1)

In [112]:
gp_HotelPrice = all_data3.groupby(['Hotel_id', 'year', 'month'], as_index=False).agg({'min_0':[np.min, np.max]})
gp_HotelPrice


Hotel_id  year month min_0       
                         amin   amax
0         1  2017    11  0.00  25.59
1         1  2017    12  0.00  36.70
2         1  2018     1  0.00  40.31
3         1  2018     2  0.00  39.39
4         1  2018     3  0.00 178.64
5         1  2018     4  0.00 211.51
6         1  2018     5  0.00 237.87
7         1  2018     6  0.00 222.91
8         1  2018     7  0.00 252.93
9         1  2018     8  0.00 251.36
10        1  2018     9  0.00 249.34
11        1  2018    10  0.00 249.54
12        1  2018    11  0.00 206.57
13        1  2018    12  0.00 192.00
14        1  2019     1  0.00 198.39
15        1  2019     2  0.00 195.97
16        1  2019     3  0.00 203.31
17        1  2019     4  0.00 217.86
18        1  2019     5  0.00 217.38
19        1  2019     6  0.00 218.87
20        1  2019     7  0.00 218.88
21        1  2019     8  0.00 218.62
22        1  2019     9  0.00   0.00
23        2  2017    11  0.00   0.00
24        2  2017    12  0.00   0.00
25        2  2018     1  0.00   0.00
26        2  2018     2  0.00   0.00
27        2  2018     3  0.00  73.99
28        2  2018     4  0.00  79.99
29        2  2018     5  0.00  89.99
..      ...   ...   ...   ...    ...
62        3  2019     3  0.00 154.99
63        3  2019     4  0.00  99.99
64        3  2019     5  0.00 109.99
65        3  2019     6  0.00 109.99
66        3  2019     7  0.00 129.99
67        3  2019     8  0.00 129.99
68        3  2019     9  0.00   0.00
69        4  2017    11  0.00   0.00
70        4  2017    12  0.00   0.00
71        4  2018     1  0.00   0.00
72        4  2018     2  0.00  59.99
73        4  2018     3  0.00  59.99
74        4  2018     4  0.00  69.99
75        4  2018     5  0.00  62.00
76        4  2018     6  0.00  79.99
77        4  2018     7  0.00  84.99
78        4  2018     8  0.00 175.00
79        4  2018     9  0.00  79.99
80        4  2018    10  0.00  67.99
81        4  2018    11  0.00  67.99
82        4  2018    12  0.00  67.99
83        4  2019     1  0.00  62.00
84        4  2019     2  0.00  62.00
85        4  2019     3  0.00  62.00
86        4  2019     4  0.00  62.00
87        4  2019     5  0.00  62.00
88        4  2019     6  0.00  62.00
89        4  2019     7  0.00  94.00
90        4  2019     8  0.00 170.00
91        4  2019     9  0.00   0.00

[92 rows x 5 columns]

In [89]:
gp_HotelPrice.columns = ['Hotel_id', 'year', 'month', 'hist_min_0', 'hist_max_0']
all_data3 = pd.merge(all_data3, gp_HotelPrice, on=['Hotel_id', 'year', 'month'], how='left')
all_data3


,Hotel_id,year,month,Date_id,LT,BKG_0,Revenue_0,BKG_1,Revenue_1,BKG_7,...,min_0,max_0,min_14,max_14,min_21,max_21,min_28,max_28,hist_min_0,hist_max_0
0,1,2018,9,354,37,2.80,310.47,2.92,291.06,2.89,...,169.94,254.68,182.85,250.75,182.35,265.55,186.01,259.64,0.00,249.34
1,1,2018,8,319,255,1.83,267.32,1.97,285.38,0.00,...,159.49,251.33,0.00,0.00,0.00,0.00,0.00,0.00,0.00,251.36
2,1,2019,1,493,298,0.00,0.00,0.00,0.00,0.00,...,106.84,246.87,106.90,256.77,100.89,246.58,102.82,260.02,0.00,198.39
3,1,2018,9,371,220,2.75,498.15,2.93,473.50,2.96,...,0.00,259.40,0.00,264.54,0.00,267.35,0.00,263.44,0.00,249.34
4,1,2018,10,387,136,0.93,178.42,0.96,167.46,0.98,...,186.09,265.91,179.19,282.99,181.92,272.82,180.83,260.72,0.00,249.54
5,1,2019,2,521,325,0.00,0.00,0.00,0.00,0.00,...,105.79,251.68,0.00,0.00,0.00,0.00,0.00,0.00,0.00,195.97
6,1,2018,11,430,360,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,206.57
7,1,2018,8,324,37,11.20,1809.55,11.08,1828.96,12.11,...,14.06,283.32,215.15,287.25,3.78,272.45,2.99,278.36,0.00,251.36
8,1,2018,7,311,94,5.48,989.52,5.76,992.50,5.66,...,193.54,266.24,184.66,258.07,179.94,229.99,183.75,264.66,0.00,252.93
9,1,2019,5,600,348,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,217.38


In [90]:
all_data3['HotelPrice_increase'] = all_data3['min_0'] - all_data3['hist_min_0']
all_data3['HotelPrice_decrease'] = all_data3['hist_max_0'] - all_data3['min_0']
all_data3

,Hotel_id,year,month,Date_id,LT,BKG_0,Revenue_0,BKG_1,Revenue_1,BKG_7,...,min_14,max_14,min_21,max_21,min_28,max_28,hist_min_0,hist_max_0,HotelPrice_increase,HotelPrice_decrease
0,1,2018,9,354,37,2.80,310.47,2.92,291.06,2.89,...,182.85,250.75,182.35,265.55,186.01,259.64,0.00,249.34,169.94,79.39
1,1,2018,8,319,255,1.83,267.32,1.97,285.38,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,251.36,159.49,91.87
2,1,2019,1,493,298,0.00,0.00,0.00,0.00,0.00,...,106.90,256.77,100.89,246.58,102.82,260.02,0.00,198.39,106.84,91.55
3,1,2018,9,371,220,2.75,498.15,2.93,473.50,2.96,...,0.00,264.54,0.00,267.35,0.00,263.44,0.00,249.34,0.00,249.34
4,1,2018,10,387,136,0.93,178.42,0.96,167.46,0.98,...,179.19,282.99,181.92,272.82,180.83,260.72,0.00,249.54,186.09,63.45
5,1,2019,2,521,325,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,195.97,105.79,90.17
6,1,2018,11,430,360,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,206.57,0.00,206.57
7,1,2018,8,324,37,11.20,1809.55,11.08,1828.96,12.11,...,215.15,287.25,3.78,272.45,2.99,278.36,0.00,251.36,14.06,237.30
8,1,2018,7,311,94,5.48,989.52,5.76,992.50,5.66,...,184.66,258.07,179.94,229.99,183.75,264.66,0.00,252.93,193.54,59.39
9,1,2019,5,600,348,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,217.38,0.00,217.38


# rolling window

In [36]:
# min_prediods= 1 =) we don't have null values ( at least
# one observation required to have a value)
# Min value 
f_min = lambda x: x.rolling(window=3, min_periods=1).min() 
# Max value
f_max = lambda x: x.rolling(window=3, min_periods=1).max()
# Mean value
f_mean = lambda x: x.rolling(window=3, min_periods=1).mean()
# Standard deviation
# std: dispersion of a dataset relative to its mean and is calculated as the square root of the variance
f_std = lambda x: x.rolling(window=3, min_periods=1).std()

function_list = [f_min, f_max, f_mean, f_std]
function_name = ['min', 'max', 'mean', 'std']
for i in range(len(function_list)):
    all_data3[('PriceComp_%s' % function_name[i])] = all_data3.groupby(['Hotel_id'])['min_0'].apply(function_list[i])

# Fill the empty std features with 0
all_data3['PriceComp_std'].fillna(0, inplace=True)

all_data3

,Hotel_id,year,month,Date_id,LT,BKG_0,Revenue_0,BKG_1,Revenue_1,BKG_7,...,min_28,max_28,hist_min_0,hist_max_0,HotelPrice_increase,HotelPrice_decrease,PriceComp_min,PriceComp_max,PriceComp_mean,PriceComp_std
0,1,2018,9,43354,37,2.80,310.47,2.92,291.06,2.89,...,186.01,259.64,0.00,249.34,169.94,79.39,169.94,169.94,169.94,0.00
1,1,2018,8,43319,255,1.83,267.32,1.97,285.38,0.00,...,0.00,0.00,0.00,251.36,159.49,91.87,159.49,169.94,164.72,7.39
2,1,2019,1,43493,298,0.00,0.00,0.00,0.00,0.00,...,102.82,260.02,0.00,198.39,106.84,91.55,106.84,169.94,145.42,33.82
3,1,2018,9,43371,220,2.75,498.15,2.93,473.50,2.96,...,0.00,263.44,0.00,249.34,0.00,249.34,0.00,159.49,88.78,81.27
4,1,2018,10,43387,136,0.93,178.42,0.96,167.46,0.98,...,180.83,260.72,0.00,249.54,186.09,63.45,0.00,186.09,97.64,93.38
5,1,2019,2,43521,325,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,195.97,105.79,90.17,0.00,186.09,97.29,93.33
6,1,2018,11,43430,360,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,206.57,0.00,206.57,0.00,186.09,97.29,93.33
7,1,2018,8,43324,37,11.20,1809.55,11.08,1828.96,12.11,...,2.99,278.36,0.00,251.36,14.06,237.30,0.00,105.79,39.95,57.45
8,1,2018,7,43311,94,5.48,989.52,5.76,992.50,5.66,...,183.75,264.66,0.00,252.93,193.54,59.39,0.00,193.54,69.20,107.91
9,1,2019,5,43600,348,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,217.38,0.00,217.38,0.00,193.54,69.20,107.91


# groupby

# lag

In [1]:
"""
all_data3 = all_data2.copy()

lag_list = [1, 2, 3]

for lag in lag_list:
    ft_name = ('HotelPrice_shifted%s' % lag)
    all_data3[ft_name] = all_data3.groupby(['Hotel_id', 'month'])['HotelPrice_14'].shift(lag)
    # Fill the empty shifted features with 0
    all_data3[ft_name].fillna(0, inplace=True)

all_data3.head()
"""

"\nall_data3 = all_data2.copy()\n\nlag_list = [1, 2, 3]\n\nfor lag in lag_list:\n    ft_name = ('HotelPrice_shifted%s' % lag)\n    all_data3[ft_name] = all_data3.groupby(['Hotel_id', 'month'])['HotelPrice_14'].shift(lag)\n    # Fill the empty shifted features with 0\n    all_data3[ft_name].fillna(0, inplace=True)\n\nall_data3.head()\n"

In [103]:
all_data3[ all_data3 == 0 ].count()

Row_id                      0
Hotel_id                    0
year                        0
month                       0
Date_id                     0
LT                       1248
BKG_0                  256406
Revenue_0              255976
BKG_1                  256994
Revenue_1              256562
BKG_7                  252237
Revenue_7              251788
BKG_14                 246557
Revenue_14             246120
HotelPrice_14           85582
BKG_21                 240757
Revenue_21             240343
HotelPrice_21           82884
BKG_28                 234796
Revenue_28             234416
HotelPrice_28           80237
Capacity                    0
Revenue_unit           297803
min_0                  144719
max_0                   48589
min_14                 130749
max_14                  47505
min_21                 124190
max_21                  46478
min_28                 118952
max_28                  45466
hist_min_0             507905
hist_max_0              27982
HotelPrice

In [71]:
all_data['Date_id']  = all_data['Date_id'] % 43000
all_data['Date_id'].sort_values()

503323     60
492721     60
181055     60
347634     60
496794     60
208476     60
354740     60
204233     60
335515     60
275397     60
297822     60
503426     60
464437     60
433144     60
300303     60
192851     60
221346     60
293435     60
209538     60
279493     60
348036     60
489681     60
239880     60
500338     60
329011     60
331617     60
279437     60
482755     60
295961     60
165242     60
         ... 
71828     734
392861    734
28589     734
259464    734
90214     734
187245    734
217318    734
324966    734
77138     734
219599    734
508869    734
150361    734
131920    734
260540    734
135232    735
280481    735
344555    735
232314    735
405613    735
206589    735
14145     735
63292     735
193148    735
181044    736
493833    736
272765    736
46435     736
153113    736
54588     736
399130    736
Name: Date_id, Length: 509602, dtype: int64

# mean stuff

In [ ]:
# Shop mean encoding
gp_hotel_mean = all_data3.groupby(['']).agg({'item_cnt_month': ['mean']})
print( gp_shop_mean ) # groupby eliminate all columns but shop_id and item_cnt_month: mean
gp_shop_mean.columns = ['shop_mean']
print( gp_shop_mean )
gp_shop_mean.reset_index(inplace=True)
print( gp_shop_mean )

train = pd.merge(train, gp_shop_mean, on=['shop_id'], how='left')
train.head()

# complete CompPrice

In [72]:
all_data[['PriceComp1_0', 'PriceComp2_0', 'PriceComp3_0' , 'PriceComp4_0']].replace(0, np.nan, inplace = True)


In [73]:
all_data['PriceComp1_0'].fillna( all_data['PriceComp1_0'].mean(), inplace=True )
all_data[['PriceComp2_0']].fillna( all_data['PriceComp2_0'].mean(), inplace=True )
all_data[['PriceComp3_0']].fillna( all_data['PriceComp3_0'].mean(), inplace=True )
all_data[['PriceComp4_0']].fillna( all_data['PriceComp4_0'].mean(), inplace=True )


In [74]:
all_data[['PriceComp1_14', 'PriceComp2_14', 'PriceComp3_14' , 'PriceComp4_14']].replace(0, np.nan, inplace = True)


In [75]:
all_data['PriceComp1_14'].fillna( all_data['PriceComp1_14'].mean(), inplace=True )
all_data[['PriceComp2_14']].fillna( all_data['PriceComp2_14'].mean(), inplace=True )
all_data[['PriceComp3_14']].fillna( all_data['PriceComp3_14'].mean(), inplace=True )
all_data[['PriceComp4_14']].fillna( all_data['PriceComp4_14'].mean(), inplace=True )


In [76]:
all_data[['PriceComp1_21', 'PriceComp2_21', 'PriceComp3_21' , 'PriceComp4_21']].replace(0, np.nan, inplace = True)


In [77]:
all_data['PriceComp1_21'].fillna( all_data['PriceComp1_21'].mean(), inplace=True )
all_data[['PriceComp2_21']].fillna( all_data['PriceComp2_21'].mean(), inplace=True )
all_data[['PriceComp3_21']].fillna( all_data['PriceComp3_21'].mean(), inplace=True )
all_data[['PriceComp4_21']].fillna( all_data['PriceComp4_21'].mean(), inplace=True )


In [78]:
all_data[['PriceComp1_28', 'PriceComp2_28', 'PriceComp3_28' , 'PriceComp4_28']].replace(0, np.nan, inplace = True)



In [79]:
all_data['PriceComp1_28'].fillna( all_data['PriceComp1_28'].mean(), inplace=True )
all_data[['PriceComp2_28']].fillna( all_data['PriceComp2_28'].mean(), inplace=True )
all_data[['PriceComp3_28']].fillna( all_data['PriceComp3_28'].mean(), inplace=True )
all_data[['PriceComp4_28']].fillna( all_data['PriceComp4_28'].mean(), inplace=True )


In [80]:
all_data.isnull().sum()

Hotel_id         0
year             0
month            0
Date_id          0
LT               0
BKG_0            0
Revenue_0        0
PriceComp1_0     0
PriceComp2_0     0
PriceComp3_0     0
PriceComp4_0     0
BKG_1            0
Revenue_1        0
BKG_7            0
Revenue_7        0
BKG_14           0
Revenue_14       0
HotelPrice_14    0
PriceComp1_14    0
PriceComp2_14    0
PriceComp3_14    0
PriceComp4_14    0
BKG_21           0
Revenue_21       0
HotelPrice_21    0
PriceComp1_21    0
PriceComp2_21    0
PriceComp3_21    0
PriceComp4_21    0
BKG_28           0
Revenue_28       0
HotelPrice_28    0
PriceComp1_28    0
PriceComp2_28    0
PriceComp3_28    0
PriceComp4_28    0
Capacity         0
dtype: int64

# seperating train and test

In [ ]:
train = all_data[:ntrain]
test = all_data[ntrain:]

In [45]:
train.shape

(355023, 39)

# random forest regression

In [118]:
#rf_features = ['shop_id', 'item_id', 'item_cnt', 'transactions', 'year',
 #              'item_cnt_mean', 'item_cnt_std', 'item_cnt_shifted1', 
  #             'shop_mean', 'item_mean', 'item_trend', 'mean_item_cnt']

"""

rf_features = all_data.columns


rf_train = X_train[rf_features]
rf_val = X_validation[rf_features]
rf_test = X_test[rf_features]
"""


'\n\nrf_features = all_data.columns\n\n\nrf_train = X_train[rf_features]\nrf_val = X_validation[rf_features]\nrf_test = X_test[rf_features]\n'

In [119]:
"""
rf_model = RandomForestRegressor(n_estimators=50, max_depth=7, random_state=0, n_jobs=-1)
rf_model.fit(train, y_train)
"""

'\nrf_model = RandomForestRegressor(n_estimators=50, max_depth=7, random_state=0, n_jobs=-1)\nrf_model.fit(train, y_train)\n'

In [120]:
"""
rf_train_pred = rf_model.predict(train)
rf_test_pred = rf_model.predict(test)

rmse = np.sqrt(mean_squared_error(rf_train_pred, y_train))
print('rmse of train : ' , rmse)
"""



"\nrf_train_pred = rf_model.predict(train)\nrf_test_pred = rf_model.predict(test)\n\nrmse = np.sqrt(mean_squared_error(rf_train_pred, y_train))\nprint('rmse of train : ' , rmse)\n"

# lightgbm

In [113]:
lgbm_model=LGBMRegressor(
        n_estimators=200,
        learning_rate=0.03,
        num_leaves=32,
        colsample_bytree=0.9497036,
        subsample=0.8715623,
        max_depth=8,
        reg_alpha=0.04,
        reg_lambda=0.073,
        min_split_gain=0.0222415,
        min_child_weight=40)



In [114]:
train = all_data[:ntrain]
test = all_data[ntrain:]


lgbm_model.fit(train, y_train)


lightgbm_train_pred = lgbm_model.predict(train)
lightgbm_test_pred = lgbm_model.predict(test)

rmse = np.sqrt(mean_squared_error(y_train, lightgbm_train_pred ))
print('rmse of train : ' , rmse)

rmse of train :  19.386822910155825


# xgboost

In [38]:
# Use only part of features on XGBoost.
#xgb_features = ['item_cnt','item_cnt_mean', 'item_cnt_std', 'item_cnt_shifted1', 
 #               'item_cnt_shifted2', 'item_cnt_shifted3', 'shop_mean', 
  #              'shop_item_mean', 'item_trend', 'mean_item_cnt']



xgb_model = XGBRegressor(max_depth=8, 
                         n_estimators=500, 
                         min_child_weight=1000,  
                         colsample_bytree=0.7, 
                         subsample=0.7, 
                         eta=0.3, 
                         seed=0)
xgb_model.fit(train, 
              y_train, 
              eval_metric="rmse", 
              eval_set=[(train, y_train), (train, y_train)], 
              verbose=20, 
              early_stopping_rounds=20)



[0]	validation_0-rmse:112.473	validation_1-rmse:112.473
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 20 rounds.
[20]	validation_0-rmse:30.0173	validation_1-rmse:30.0173
[40]	validation_0-rmse:24.2858	validation_1-rmse:24.2858
[60]	validation_0-rmse:23.069	validation_1-rmse:23.069
[80]	validation_0-rmse:22.5709	validation_1-rmse:22.5709
[100]	validation_0-rmse:22.1899	validation_1-rmse:22.1899
[120]	validation_0-rmse:21.8132	validation_1-rmse:21.8132
[140]	validation_0-rmse:21.3873	validation_1-rmse:21.3873
[160]	validation_0-rmse:20.9891	validation_1-rmse:20.9891
[180]	validation_0-rmse:20.7108	validation_1-rmse:20.7108
[200]	validation_0-rmse:20.4741	validation_1-rmse:20.4741
[220]	validation_0-rmse:20.2261	validation_1-rmse:20.2261
[240]	validation_0-rmse:20.0595	validation_1-rmse:20.0595
[260]	validation_0-rmse:19.8489	validation_1-rmse:19.8489
[280]	validation_0-rmse:19.6904	valid

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, eta=0.3, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=8, min_child_weight=1000, missing=None,
       n_estimators=500, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=0, silent=True, subsample=0.7)

In [127]:
"""
xgb_train_pred = xgb_model.predict(train)

xgb_test_pred = xgb_model.predict(test)

rmse = np.sqrt(mean_squared_error(xgb_train_pred, y_train))
print('rmse of train : ' , rmse)
"""

"\nxgb_train_pred = xgb_model.predict(train)\n\nxgb_test_pred = xgb_model.predict(test)\n\nrmse = np.sqrt(mean_squared_error(xgb_train_pred, y_train))\nprint('rmse of train : ' , rmse)\n"

# lasso

In [40]:

lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))


lasso.fit(train, y_train)

y_lasso_pred = lasso.predict(train)

rmse = np.sqrt(mean_squared_error(y_lasso_pred , y_train))
print(rmse)


38.14938177319193


# ENet

In [129]:
"""
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))

ENet.fit(train, y_train)

y_ENet_pred = ENet.predict(train)

rmse = np.sqrt(mean_squared_error(y_ENet_pred , y_train))
print(rmse)
"""


'\nENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))\n\nENet.fit(train, y_train)\n\ny_ENet_pred = ENet.predict(train)\n\nrmse = np.sqrt(mean_squared_error(y_ENet_pred , y_train))\nprint(rmse)\n'

# KRR

In [ ]:

KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)

KRR.fit(train, y_train)

y_KRR_pred = KRR.predict(train)

rmse = np.sqrt(mean_squared_error(y_KRR_pred , y_train))
print(rmse)


In [ ]:
y_KRR_pred = KRR.predict(test)


# Gboost

In [131]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

In [132]:
"""
GBoost.fit(train, y_train)


y_GBoost_pred = GBoost.predict(train)

rmse = np.sqrt(mean_squared_error(y_GBoost_pred , y_train))
print(rmse)
"""

'\nGBoost.fit(train, y_train)\n\n\ny_GBoost_pred = GBoost.predict(train)\n\nrmse = np.sqrt(mean_squared_error(y_GBoost_pred , y_train))\nprint(rmse)\n'

# submission

In [58]:
lightgbm_train_pred.shape

(356720,)

In [93]:
prediction_df = pd.DataFrame(test_id, columns=['Row_id'])
prediction_df['HotelPrice_0'] = lightgbm_test_pred

#prediction_df['sales'] = rf_test_pred
#prediction_df['sales'] = lr_test_pred
#prediction_df['HotelPrice_0'] = xgb_test_pred
#prediction_df['sales'] = dmatrix_test_pred
#prediction_df['sales'] = y_lasso_pred
#prediction_df['sales'] = y_ENet_pred
#prediction_df['sales'] = y_KRR_pred
#prediction_df['sales'] = y_GBoost_pred
#prediction_df['sales'] = y_stacked_averaged_models_pred
#prediction_df['sales'] = lgbdataset_pred_val


train_df = pd.DataFrame()
train_df['y_train'] = y_train
train_df['prediction'] = lightgbm_train_pred


prediction_df.to_csv(r'C:\Users\pc\Desktop\Data Science Folder\Infor\submission.csv', index=False)
train_df.to_csv(r'C:\Users\pc\Desktop\Data Science Folder\Infor\whatsproblem.csv', index=False)

train_df.head(10)

,y_train,prediction
0,209.12,208.33
1,189.50,175.32
2,165.19,166.54
3,170.13,184.67
4,184.63,195.30
5,158.10,161.07
6,0.00,1.56
7,243.88,247.19
8,193.68,199.79
9,0.00,1.04
